<a href="https://colab.research.google.com/github/agu3ro/Pytorch-DL/blob/main/07_07_SuperGradients_Image_Classification_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!wget https://dissect.csail.mit.edu/datasets/miniplaces.zip --no-check-certificate && unzip miniplaces.zip

# 👋🏽 What's up! It's [Harpreet](https://twitter.com/DataScienceHarp)

I'll be guiding you through this notebook. At any point, if you get stuck or have questions, there are three ways to get in touch:

1) Send me an email with your issue: harpreet.sahota@deci.ai

2) Hop into the [Deep Learning Daily (powered by Deci) Discord server](https://discord.gg/p9ecgRhDR8), and let me know what your question is.

3) [Open an issue on GitHub](https://github.com/Deci-AI/super-gradients/issues/new/choose)

---

# 🖼️ Image Classification Project Template with SuperGradients

This notebook template is for data that is already in ImageFolder format.

If you are unfamiliar with ImageFolder format, it looks like this:

```
├── train
│   ├── class1
|      ├── 1.jpg
│      ├── 2.jpg
│   ├── class2
|      ├── 1.jpg
│      ├── 2.jpg
├── valid
│   ├── class1
|      ├── 1.jpg
│      ├── 2.jpg
│   ├── class2
|      ├── 1.jpg
│      ├── 2.jpg
```

With parent folder repeating for validtaion and testing data.

In [7]:
%%capture
!pip install super_gradients==3.2.0

In [12]:
import os
from pathlib import Path

import torch
from torchvision import datasets, transforms

import super_gradients
from super_gradients.common.object_names import Models
from super_gradients.training import Trainer, training_hyperparams, models

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

# ⚒️ Config

This holds variables for the notebook.

You will define the model, training params, image type, number of classes, and
relevant directories in this class.



If you have a question you can leave a comment on this notebook, or visit the community and post it in the [Q&A section](https://www.deeplearningdaily.community).


In [ ]:
class config:
    # specify the paths to datasets
    ROOT_DIR = Path()
    TRAIN_DIR = ROOT_DIR.joinpath('miniplaces/train')
    TEST_DIR = ROOT_DIR.joinpath('miniplaces/test')
    VAL_DIR = ROOT_DIR.joinpath('miniplaces/val')

    CHECKPOINT_DIR = 'checkpoints'
    EXPERIMENT_NAME = "YOUR-EXPERIMENT-NAME"
    PRETRAINED_WEIGHTS = 'imagenet'

    # note if you're averaging the best model then replace "ckpt_best.pth" with "average_model.pth"
    CHECKPOINT_TYPE = "ckpt_best.pth"

    # set the input height and width
    INPUT_HEIGHT = 224
    INPUT_WIDTH = 224

    # what mean and std do you want to standardize to
    NORM_MEAN = [0.485, 0.456, 0.406]
    NORM_STD = [0.229, 0.224, 0.225]

    BATCH_SIZE = 32
    MODEL_NAME = "CHOOSE-YOUR-MODEL"
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    TRAINING_PARAMS = 'training_hyperparams/default_train_params'
    LOSS = "cross_entropy"
    OPTIMIZER = 'Adam'
    EPOCHS = 100
    INITIAL_LR = 3e-4

    NUM_CLASSES = 100

    METRICS_LIST = ['Accuracy', 'Top5']

# 💾 Dataset and Dataloader

You can experiment with any transforms you'd like. [Here](https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py) are the torchvision transforms you can use.

In [ ]:
# define the transform
train_transform = transforms.Compose([
    transforms.Resize((config.INPUT_HEIGHT, config.INPUT_WIDTH)),
    transforms.ToTensor(),  # convert image to PyTorch tensor
    transforms.Normalize(mean=config.NORM_MEAN, std=config.NORM_STD)
])

test_transform = transforms.Compose([
    transforms.Resize((config.INPUT_HEIGHT, config.INPUT_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.NORM_MEAN, std=config.NORM_STD)
])

# define the datasets
train_dataset = datasets.ImageFolder(config.TRAIN_DIR,
                                     transform=train_transform
                                     )

val_dataset = datasets.ImageFolder(config.VAL_DIR,
                                   transform=test_transform
                                   )

test_dataset = datasets.ImageFolder(config.TEST_DIR,
                                    transform=test_transform
                                    )

# define the dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.BATCH_SIZE,
                                           shuffle=True
                                           )

val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=config.BATCH_SIZE,
                                         shuffle=False
                                         )

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=config.BATCH_SIZE,
                                          shuffle=False
                                          )

FileNotFoundError: [Errno 2] No such file or directory: 'miniplaces/test'

# 🎛️ Training Params

You'll use the [default training parameters](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml) for this notebook.

In [ ]:
training_params =  training_hyperparams.get(config.TRAINING_PARAMS)

In [ ]:
training_params

{'resume': False,
 'resume_path': None,
 'resume_from_remote_sg_logger': False,
 'ckpt_name': 'ckpt_latest.pth',
 'lr_mode': None,
 'lr_schedule_function': None,
 'lr_warmup_epochs': 0,
 'lr_warmup_steps': 0,
 'lr_cooldown_epochs': 0,
 'warmup_initial_lr': None,
 'step_lr_update_freq': None,
 'cosine_final_lr_ratio': 0.01,
 'warmup_mode': 'linear_epoch_step',
 'lr_updates': [],
 'pre_prediction_callback': None,
 'optimizer': 'SGD',
 'optimizer_params': {},
 'load_opt_params': True,
 'zero_weight_decay_on_bias_and_bn': False,
 'loss': None,
 'criterion_params': {},
 'ema': False,
 'ema_params': {'decay': 0.9999, 'decay_type': 'exp', 'beta': 15},
 'train_metrics_list': [],
 'valid_metrics_list': [],
 'metric_to_watch': 'Accuracy',
 'greater_metric_to_watch_is_better': True,
 'launch_tensorboard': False,
 'tensorboard_port': None,
 'tb_files_user_prompt': False,
 'save_tensorboard_to_s3': False,
 'precise_bn': False,
 'precise_bn_batch_size': None,
 'sync_bn': False,
 'silent_mode': False

In [ ]:
training_params['loss'] = config.LOSS
training_params['optimizer'] = config.OPTIMIZER
training_params['max_epochs'] = config.EPOCHS
training_params['train_metrics_list'] = config.METRICS_LIST
training_params['valid_metrics_list'] = config.METRICS_LIST
training_params['initial_lr'] = config.INITIAL_LR

In [ ]:
training_params

{'resume': False,
 'resume_path': None,
 'resume_from_remote_sg_logger': False,
 'ckpt_name': 'ckpt_latest.pth',
 'lr_mode': None,
 'lr_schedule_function': None,
 'lr_warmup_epochs': 0,
 'lr_warmup_steps': 0,
 'lr_cooldown_epochs': 0,
 'warmup_initial_lr': None,
 'step_lr_update_freq': None,
 'cosine_final_lr_ratio': 0.01,
 'warmup_mode': 'linear_epoch_step',
 'lr_updates': [],
 'pre_prediction_callback': None,
 'optimizer': 'SGD',
 'optimizer_params': {},
 'load_opt_params': True,
 'zero_weight_decay_on_bias_and_bn': False,
 'loss': 'cross_entropy',
 'criterion_params': {},
 'ema': False,
 'ema_params': {'decay': 0.9999, 'decay_type': 'exp', 'beta': 15},
 'train_metrics_list': ['Accuracy', 'Top5'],
 'valid_metrics_list': ['Accuracy', 'Top5'],
 'metric_to_watch': 'Accuracy',
 'greater_metric_to_watch_is_better': True,
 'launch_tensorboard': False,
 'tensorboard_port': None,
 'tb_files_user_prompt': False,
 'save_tensorboard_to_s3': False,
 'precise_bn': False,
 'precise_bn_batch_size':

There are a few things you can try to see how you fare: try a different optimizer (you can the optimizer by using a passing of the following strings "Adam", "AdamW", "SGD", or "RMSProp".

# 👩🏽‍🦳 Instantiate model



In [ ]:
model = models.get(config.MODEL_NAME,
                   num_classes = config.NUM_CLASSES,
                   pretrained_weights=config.PRETRAINED_WEIGHTS)

# 🏋🏾 Instantiate trainer

In [ ]:
trainer = Trainer(ckpt_root_dir=config.CHECKPOINT_DIR,
                  experiment_name=config.EXPERIMENT_NAME
                  )

# 🦾 Train model

In [ ]:
trainer.train(model=model,
              training_params=training_params,
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

# 🙌🏼 Load best model

In [ ]:
best_model = models.get(config.MODEL_NAME,
                        num_classes=config.NUM_CLASSES,
                        checkpoint_path=os.path.join(full_model_trainer.checkpoints_dir_path,
                                                     config.CHECKPOINT_TYPE)
                        )

# 🧐 Evaluate on test set

In [ ]:
trainer.test(model=best_model,
             test_loader=test_dataloader,
             test_metrics_list=config.METRICS_LIST
             )

# 🔮 Predicting with the best model

In [ ]:
from super_gradients.training.processing.processing import default_imagenet_processing_params
processing_params = default_imagenet_processing_params()
processing_params['class_names'] = train_dataset.classes
best_model.set_dataset_processing_params(**processing_params)

In [ ]:
img_url = 'ENTER-SOME-URL-HERE'
best_model.predict(img_url).show()

# Your homework

Copy/fork this notebook and try some different architectures.

If you have a question you can leave a comment on this notebook, or visit the community and post it in the [Q&A section](https://www.deeplearningdaily.community).

## Use a different pretrained model

You can change the model you use. Take a look at the [SG model zoo](https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/model_zoo.md)

For example, if you wanted to use RegNet you would do the following:

```
resnet_imagenet_model = models.get(model_name='regnetY800', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_regnetY_train_params')
```

Note you can also pass 'model_name=regnetY200', 'model_name=regnetY400', 'model_name=regnetY600' to try a variety of the architecture

For ResNet50, you would do:

```
resnet_imagenet_model = models.get(model_name='resnet50', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_resnet50_train_params')
```

Note you can also pass 'model_name=resnet18' or 'model_name=resnet34' to try a variety of the architecture

For MobileNetV2, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v2', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv2_train_params')
```

For MobileNetV3, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v3_large', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv3_train_params')
```

Note you can also pass 'model_name=mobilenet_v3_small' to try a variety of the architecture


For ViT, you would do:


```
vit_imagenet_model = models.get(model_name='vit_base', num_classes=NUM_CLASSES, pretrained_weights='imagenet')
vit_params =  training_hyperparams.get("training_hyperparams/imagenet_vit_train_params")
```

Note you can also pass 'model_name=vit_large' to try a variety of the architecture


I encourage you play around with different optimizers, all you have to do is change the value of `training_params["optimizer"]`. You can use one of ['Adam','SGD','RMSProp'] out of the box. You can play around with the optimizer params as well.

In general, play and tweak around the training recipies...

## Training recipes

SuperGradients has a number of [training recipes](https://github.com/Deci-AI/super-gradients/tree/master/src/super_gradients/recipes) you can use. [See here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml) for more information about the training params.